In [1]:
# potrzebne bilioteki
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input,State
from dash_bootstrap_components._components.Container import Container
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from itertools import chain

In [2]:
# wczytanie danych
df=pd.read_excel(r'C:\Users\natal\OneDrive\Dokumenty\AnalizaDanych\DATA\1.1.Wypadki_ogolem_smiertleni_ranni_POW.xlsx')
df_Wypadki=df.groupby(["Jednostka", "Rok"], as_index = False).sum()

df2=pd.read_excel(r'C:\Users\natal\OneDrive\Dokumenty\AnalizaDanych\DATA\3.Wypadki_wg_przyczyn_WOJ.xlsx')
df2 = df2[ df2['Jednostka'] != 'POLSKA']
df2 = df2.drop(['Wina kierujących pojazdami ogółem', 'Wina pieszych - ogółem'], axis=1)
df_Przyczyny=df2.groupby(["Jednostka","Rok"], as_index = False).sum()

# mapowanie kolorów województw 
c = dict(zip(df_Wypadki["Jednostka"].unique(), px.colors.qualitative.G10))

In [3]:
external_stylesheets = [dbc.themes.FLATLY]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

In [4]:
#dodanie navbar
navbar = dbc.NavbarSimple(
    brand="Analiza i wizualizacja danych dotyczących wypadków drogowych w Polsce w latach 2014-2020",
    brand_href="/",
    color="primary",
    dark=True,
)


app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    navbar,
    html.Div(id='page-content')
])

#definicja poszczegolnych kolumn
def colGen(href, header, buttonLabel="Kliknij"):
    cardStyle = {
        'height': '100%',
        'minHeight': '180px', 
        'display': 'flex', 
        'border': 'none',
        'borderRadius': '5px',
        'boxShadow': '5px 0 10px 10px #f7f7f7',
        'justifyContent': 'space-between'
    }
    headerStyle = {'margin': 'auto'}

    return dbc.Col(
        dbc.Card(
            children=[
                html.H3(
                    children=header,
                    className="text-center",
                    style=headerStyle
                ),
                dbc.Button(
                    buttonLabel,
                    href=href,
                    color="primary",
                    className="mt-3"),
                #dcc.Link('Kliknij', href=href ,style={'font-family': 'Lato',
                                                     #'font-weight': 'bold', 'text-align':'center','font-size':'1.5em'})
                
            ],
            body=True, 
            color="dark", 
            outline=True,
            style=cardStyle
        ),
        width = 4, 
        className="mb-4"
    )

index_page = html.Div([
    dbc.Container([
        dbc.Row([
            dbc.Col(
                html.H1(
                    "Projekt na kierunku Data Scientist. Analityk danych", 
                    className="text-center"
                ), 
                className="mb-5 mt-5"
            )
        ]),
        dbc.Row([
            colGen('/page-1', 'Wizualizacja wypadków ogółem'),
            colGen('/page-2', 'Wizualizacja wybranych przyczyn'),
            colGen('/page-3', 'Podstawowe statystyki')          
        ]),
        dbc.Row([
            dbc.Col(
                html.H4(
                    children='''Praca ta opiera się na analizie i wizualizacji danych dotyczących wypadków drogowych 
                                w Polsce w latach 2014-2020 z wykorzystaniem bibliotek Dash i Plotly.
                                Dane do projektu pobrane zostały z Banku Danych Lokalnych(bdl.stat.gov.p) - 
                                największej w Polsce bazy danych o gospodarce, społeczeństwie i środowisku.'''
                ), 
                className="mb-4")
        ]),
         dbc.Row([
            dbc.Col(
                html.H5(
                    children='Główne cele prezentowanej analizy:',
                    style={"font-weight": "bold"}
                ), 
                className="mb-4")
        ]),
        dbc.Row([
            dbc.Col(
                html.H5(
                    children='- ogólne przedstawienie zjawiska,'
                ), 
                className="mb-4")
        ]),
        dbc.Row([
            dbc.Col(
                html.H5(
                    children='- identyfikacja najbardziej niebezpiecznego województwa w Polsce pod względem wypadków drogowych,'
                ), 
                className="mb-4")
        ]),
        dbc.Row([
            dbc.Col(
                html.H5(
                    children='- przedstawienie wybranych przyczyn,'
                ), 
                className="mb-4")
        ]),
        dbc.Row([
            dbc.Col(
                html.H5(
                    children='- ocena zjawiska w czasie.'
                ), 
                className="mb-4")
        ]),
      dbc.Row([
            dbc.Col(
                html.H5(
                    children='Wnioski:',
                    style={"font-weight": "bold"}
                ), 
                className="mb-4")
        ]),
        dbc.Row([
            dbc.Col(
                html.H5(
                    children='- W analizowanym przedziale czasowym obserwuje się spadek liczby wypadków drogowych.'
                ), 
                className="mb-4")
        ]),
        dbc.Row([
            dbc.Col(
                html.H5(
                    children='''- W latach 2014-2020 w Polsce miało miejsce 219863 wypadków drogowych z czego rannych 261854 osób. Liczba ofiar śmiertelnych w tych latach
                                wyniosła 20259.
                                '''
                ), 
                className="mb-4")
        ]),
        dbc.Row([
            dbc.Col(
                html.H5(
                    children='''- W latach 2014-2020 najwięcej wypadków ogółem odnotowano w woj. Mazowieckim (27313), najmniej w
                                woj. Podlaskim (4448).
                                '''
                ), 
                className="mb-4")
        ]),
        dbc.Row([
            dbc.Col(
                html.H5(
                    children='- Najwięcej wypadków miało miejsce w woj. Mazowieckim w roku 2014, najmniej w woj.Podlaskim w roku 2020.'
                            
                ), 
                className="mb-4")
        ]),
        dbc.Row([
            dbc.Col(
                html.H5(
                    children='- W latach 2014-2020 najwięcej ofiar śmiertelnych w woj.Mazowieckim (3314), najmniejsza liczba w woj.Opolskim (602).'
                ), 
                className="mb-4")
        ]),
        dbc.Row([
            dbc.Col(
                html.H5(
                    children='- Najwięcej ofiar śmiertelnych odnotowano w woj. Mazowieckim w roku 2014, najmniej w woj.Opolskim w roku 2020.'
                ), 
                className="mb-4")
        ]),
         dbc.Row([
            dbc.Col(
                html.H5(
                    children='- W latach 2014-2020 najwięcej rannych w woj. Mazowieckim (31958), najmniejsza liczba w woj.Podlaskim (5135).'
                ), 
                className="mb-4")
        ]),
         dbc.Row([
            dbc.Col(
                html.H5(
                    children='- Najwięcej rannych w woj.Śląskim w roku 2014, najmniej w woj.Podlaskim w roku 2020.'
                ), 
                className="mb-4")
        ]),
         dbc.Row([
            dbc.Col(
                html.H5(
                    children='- Od roku 2016 do 2019 w woj. Wielkopolskim następuje znaczny wzrost ogólnej liczby wypadków.'
                ), 
                className="mb-4")
        ]),
         dbc.Row([
            dbc.Col(
                html.H5(
                    children='''- W badanych latach więcej wypadków spowodowanych było przez kierujących pojazdami niż pieszych.
                                  Do głównych przyczyn wypadków z winy kierowców należą nieprzestrzeganie pierwszeństwa przejazdu oraz
                                  niedostosowanie prędkości do warunków ruchu. Najmniej wypadków odnotowuje się w wyniku nieprawidłowego wyprzedzania.
                                  
                             '''
                ), 
                className="mb-4")
        ]),
        dbc.Row([
            dbc.Col(
                html.H5(
                    children='- Najczęstszą winą pieszych było nieostrożne wejście na jezdnię.'
                ), 
                className="mb-4")
        ]),
         dbc.Row([
            dbc.Col(
                html.H5(
                    children='Znaczenie projektu:',
                    style={"font-weight": "bold"}
                ), 
                className="mb-4")
        ]),
        dbc.Row([
            dbc.Col(
                html.H5(
                    children=''' - Analizowane dane mogą być pomocne w poprawie bezpieczeństwa na drogach. Dzięki nim można lepiej zidentyfikować
                                obszary o wysokim stopniu zagrożenia wypadkami.
                                '''
                ), 
                className="mb-4")
        ]),
        
    ]),
])


        
######################### STRONA 1 ################################

page_1_layout = html.Div([
        html.Div([
            dcc.Dropdown(id='dpdn2', value=df_Wypadki['Jednostka'].unique(), multi=True,
                         options=[{'label': x, 'value': x} for x in df_Wypadki.Jednostka.unique()])
        ]),
        html.H3('Wizualizacja wypadków ogółem'),
        html.Div([
            dcc.Graph(id='my-graph', figure={}, clickData=None, hoverData=None,  
                      config={
                          'staticPlot': False,     
                          'scrollZoom': True,     
                          'doubleClick': 'reset',  
                          'showTips': False,       
                          'displayModeBar': True,  
                          'watermark': True,},
                      className='' #six columns
                      ),
        ]),
        html.Div(
            dbc.Row(
                [
                    dbc.Col(dcc.Graph(id='pie-graph', figure={})),
                    dbc.Col(dcc.Graph(id='pie-graph2', figure={}))
                ]
            )
        ),
    html.Div(id='page-1-content'),
    html.Br(),
    dcc.Link('Go to Page 2', href='/page-2'),
    html.Br(),
    dcc.Link('Go back to home', href='/')
])

######################### STRONA 2 ################################

page_2_layout = html.Div([
    html.Div([
            dcc.Dropdown(id='dpdn3', value='DOLNOŚLĄSKIE', multi=True,
                         options=[{'label': y, 'value': y} for y in df_Przyczyny.Jednostka.unique()])
    ]),
    html.H3('Wizualizacja wybranych przyczyn'),
    html.Div(id='page-2-content'),
    html.Div([dcc.Graph(id='line_graph')],className='six columns'),
    html.Div([html.Label(['Wybierz przyczynę wypadku:'],style={'text-align':'right'})]),
    html.Div([
            dcc.Dropdown(id='dpdn4', value='Wina kierujących pojazdami - niedostosowanie prędkości do warunków ruchu', multi=True,
                         options=[{'label': y, 'value': y} for y in df_Przyczyny.columns[2:]])
    ]),
    dcc.Link('Analiza wypadków', href='/page-1'),
    #html.Br(),
    dcc.Link('Go back to home', href='/')
])

# Update the index
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/page-1':
        return page_1_layout
    elif pathname == '/page-2':
        return page_2_layout
    else:
        return index_page

In [5]:
@app.callback(
    Output(component_id='my-graph', component_property='figure'),
    Input(component_id='dpdn2', component_property='value')
)

def update_graph(Jednostka_chosen):
    dff = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
    fig = px.line(data_frame=dff, x='Rok', y='Wypadki ogółem', color='Jednostka',
                  custom_data=['Jednostka', 'Wypadki ogółem', 'Ofiary śmiertelne', 'Ranni'],
                 title=f'Wypadki ogółem: ', color_discrete_map=c)
    fig.update_traces(mode='lines+markers')
    return fig

In [6]:
@app.callback(
    Output(component_id='pie-graph', component_property='figure'),
    Input(component_id='my-graph', component_property='hoverData'),
    Input(component_id='my-graph', component_property='clickData'),
    Input(component_id='my-graph', component_property='selectedData'),
    Input(component_id='dpdn2', component_property='value')
)
def update_side_graph(hov_data, clk_data, slct_data, Jednostka_chosen):
    if hov_data is None:
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        dff2 = df_Wypadki[df_Wypadki.Rok == 2014]
        fig2 = px.pie(data_frame=dff2, values='Ofiary śmiertelne', names='Jednostka',
                      title='Ofiary śmiertelne w 2014',color="Jednostka", color_discrete_map=c)
        return fig2
    else:
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        hov_year = hov_data['points'][0]['x']
        dff2 = dff2[dff2.Rok == hov_year]
        fig2 = px.pie(data_frame=dff2, values='Ofiary śmiertelne', names='Jednostka',
                      title=f'Ofiary śmiertelne w: {hov_year}',color='Jednostka', color_discrete_map=c)
        return fig2
    
@app.callback(
    Output(component_id='pie-graph2', component_property='figure'),
    Input(component_id='my-graph', component_property='hoverData'),
    Input(component_id='my-graph', component_property='clickData'),
    Input(component_id='my-graph', component_property='selectedData'),
    Input(component_id='dpdn2', component_property='value')
)
def update_side_graph(hov_data, clk_data, slct_data, Jednostka_chosen):
    if hov_data is None:
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        dff2 = df_Wypadki[df_Wypadki.Rok == 2014]
        fig2 = px.pie(data_frame=dff2, values='Ranni', names='Jednostka',
                      title='Ranni w 2014',color='Jednostka', color_discrete_map=c)
        return fig2
    else:
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        hov_year = hov_data['points'][0]['x']
        dff2 = dff2[dff2.Rok == hov_year]
        fig2 = px.pie(data_frame=dff2, values='Ranni', names='Jednostka',
                      title=f'Ranni w: {hov_year}',color='Jednostka', color_discrete_map=c)
        return fig2

In [7]:
#### callback do strony 2###
@app.callback(
    Output(component_id='line_graph', component_property='figure'),
    Input(component_id='dpdn3', component_property='value'),
    Input(component_id='dpdn4', component_property='value')
)
def build_graph(dpdn3,dpdn4):
    dpdn3List=list({dpdn3})
    dpdn4List = list({'Jednostka','Rok'})+list({dpdn4})
    print(dpdn3List)
    print(dpdn4List)
    dff = df_Przyczyny.loc[:,dpdn4List]
    dff2=dff.loc[dpdn3List,:]
    print(dff2)

In [8]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Dec/2021 09:06:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 09:06:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 09:06:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 09:06:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 09:06:19] "POST /_dash-update-component HTTP/1.1" 200 -


['DOLNOŚLĄSKIE']
['Rok', 'Jednostka', 'Wina kierujących pojazdami - niedostosowanie prędkości do warunków ruchu']
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/dominika.lisiecka/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/dominika.lisiecka/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/dominika.lisiecka/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/dominika.lisiecka/opt/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/dominika.lisiecka/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/dominika.lisiecka/

127.0.0.1 - - [30/Dec/2021 09:06:20] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [30/Dec/2021 09:06:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 09:06:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 09:06:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 09:06:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 09:06:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 09:06:26] "POST /_dash-update-component HTTP/1.1" 200 -
